In [30]:

import pandas as pd
import anndata as ad
import episcanpy.api as epi
import numpy as np
import pysam
import gzip

In [31]:
GTF_PATH = '/home/rstudio/snapATAC_Analysis/gencode.v39.annotation.gtf'  # gtf file to use for peaks annotation
#INPUT_PATH = '/home/rstudio/data/anndata'  # path where the h5ad object is saved
WORKING_DIR = '/mnt/workspace_stud/catlas_ref/'
INPUT = WORKING_DIR + 'cellXcCRE/'
OUTPUT_PATH = '/home/rstudio/testdata'  # path where processed adata file can be save
BAM_FILE = '/mnt/workspace_stud/jdetlef/ENC-1JKYN-191-SM-JF1O8_snATAC_colon_sigmoid_Rep1.bam'  # path to bam file
FILE_NAME = 'Human_brain_1'
H5AD = FILE_NAME + '.h5ad'
FRAG_FILE = WORKING_DIR + 'frag/' + FILE_NAME + '.bed.gz'  # path to fragments file
#H5AD = 'ENC-1JKYN-146-SM-A8CPH_snATAC_esophagus_muscularis_mucosa_Rep1.h5ad'  # name of the h5ad file

In [32]:
adata = epi.read_h5ad(INPUT + H5AD)
adata

AnnData object with n_obs × n_vars = 6717 × 1154611
    obs: 'logUMI', 'tsse', 'tissue', 'cell type', 'Life stage', 'closest Cell Ontology term(s)', 'Cell Ontology ID'
    var: 'Chromosome', 'hg38_Start', 'hg38_End', 'Class', 'Present in fetal tissues', 'Present in adult tissues', 'CRE module'

In [61]:
data = adata.var[['hg38_Start', 'hg38_End']]
adata.obs

,logUMI,tsse,tissue,cell type,Life stage,closest Cell Ontology term(s),Cell Ontology ID
index,,,,,,,
Human_brain_1_1+AGCGATAGAACCAGGTAATGACGTGTACTGAC,3.701827,13.956466,Human_brain_1,Microglia,Adult,microglial cell,CL:0000129
Human_brain_1_1+AGCGATAGAACCAGGTTTACGACCAGGCGAAG,3.334454,15.030303,Human_brain_1,Microglia,Adult,microglial cell,CL:0000129
Human_brain_1_1+AGCGATAGAAGCAACGTCCGGTAAGTACTGAC,3.506234,18.361836,Human_brain_1,Microglia,Adult,microglial cell,CL:0000129
Human_brain_1_1+AGCGATAGAAGCAACGTTGGAAGTCAGGACGT,3.156246,8.838384,Human_brain_1,Microglia,Adult,microglial cell,CL:0000129
Human_brain_1_1+AGCGATAGACGAATCTAAGAGATGCCTATCCT,3.318481,13.131313,Human_brain_1,Microglia,Adult,microglial cell,CL:0000129
...,...,...,...,...,...,...,...
Human_brain_1_1+TCCGGAGATCTATCCGATAGCCTTCAGGACGT,3.042182,20.875421,Human_brain_1,Astrocyte 1,Adult,astrocyte,CL:0000127
Human_brain_1_1+TCCGGAGATTAGCCTCAATGACGTGGCTCTGA,3.172603,18.181818,Human_brain_1,Astrocyte 1,Adult,astrocyte,CL:0000127
Human_brain_1_1+TCCGGAGATTAGCCTCTTACGACCAGGCGAAG,3.415140,11.588412,Human_brain_1,Astrocyte 1,Adult,astrocyte,CL:0000127


In [59]:
f = gzip.open(WORKING_DIR + '/frag/colon_sigmoid_SM-JF1O8_rep1_fragments.bed.gz', "rt")

for index in data.index:
    f_start = data["hg38_Start"][index]
    f_end = data["hg38_End"][index]
    for line in f:
        columns = line.rstrip().split("\t")
        barcode = columns[3]
        start = int(columns[1])
        end = int(columns[2])
        
        if f_start <= start and f_end >= end:
            print("Hit: " + barcode)

Hit: AAACTACCAGAAAGCACTAGCG
Hit: AAACTACCAGAAAGCACTAGCG
Hit: AAACTACCAGAACCATGCATGA
Hit: AAACTACCAGAACCATGCATGA
Hit: AAACTACCAGACACGCGTGAAC
Hit: AAACTACCAGACACGCGTGAAC
Hit: AAACTACCAGACACGCGTGAAC
Hit: AAACTACCAGACATCCGGTCTC
Hit: AAACTACCAGACATCTAAGGAG
Hit: AAACTACCAGACATCTAAGGAG
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACATGCCGTGCT
Hit: AAACTACCAGACCAATCGCTTG
Hit: AAACTACCAGACCAATCGCTTG
Hit: AAACTACCAGACCACGCAAACT
Hit: AAACTACCAGACCACGCAAACT
Hit: AAACTACCAGACCCGAGTCAGT
Hit: AAACTACCAGACCCGATGGCTA
Hit: AAACTACCAGACCCGATGGCTA
Hit: AAACTACCAGACCCGATGGCTA
Hit: AAACTACCAGACCCGATGGCTA
Hit: AAACTACCAGACCCGATGGCTA
Hit: AAACTACCAGACGGT

Hit: AACTGCGCCAATTAGGAGGTCT
Hit: AACTGCGCCAATTAGGAGGTCT
Hit: AACTGCGCCAATTAGGAGGTCT
Hit: AACTGCGCCACAACCACGAGTG
Hit: AACTGCGCCACAACCACGAGTG
Hit: AACTGCGCCACAATCTGGGTGT
Hit: AACTGCGCCACGCCGAATTACT
Hit: AACTGCGCCACGCCGAATTACT
Hit: AACTGCGCCACGTAGGATTCTG
Hit: AACTGCGCCACGTTTATCACAC
Hit: AACTGCGCCACGTTTATCACAC
Hit: AACTGCGCCACTAATCCAGATG
Hit: AACTGCGCCACTCCGATATGAC
Hit: AACTGCGCCACTCCGATATGAC
Hit: AACTGCGCCACTGTTGGTCCTT
Hit: AACTGCGCCACTGTTGGTCCTT
Hit: AACTGCGCCAGACCAGACCTGT
Hit: AACTGCGCCAGACCAGACCTGT
Hit: AACTGCGCCAGACGTCCCTCAT
Hit: AACTGCGCCAGACGTCCCTCAT
Hit: AACTGCGCCAGACGTCCCTCAT
Hit: AACTGCGCCAGACGTCCCTCAT
Hit: AACTGCGCCAGATACTTGCTGG
Hit: AACTGCGCCAGATACTTGCTGG
Hit: AACTGCGCCAGCTTCGGAGTCT
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTACAGCTTC
Hit: AACTGCGCCAGGGTTGAACCTA
Hit: AACTGCGCCAGGGTTGAACCTA
Hit: AACTGCGCCAGGGTTGAACCTA
Hit: AACTGCGCCAGGGTT

Hit: AAGCTATACCAGACTAGGAGGA
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATACCTGGAG
Hit: AAGCTATACCAGATCGTTTGGC
Hit: AAGCTATACCAGATCGTTTGGC
Hit: AAGCTATACCAGATCGTTTGGC
Hit: AAGCTATACCAGATCGTTTGGC
Hit: AAGCTATACCAGATCGTTTGGC
Hit: AAGCTATACCAGCGAATAACCC
Hit: AAGCTATACCAGGAGTCGGAGA
Hit: AAGCTATACCAGGCGGCATTAC
Hit: AAGCTATACCAGTCCTGGTACT
Hit: AAGCTATACCAGTCTGTGTAGA
Hit: AAGCTATACCAGTCTGTGTAGA
Hit: AAGCTATACCAGTGTCAGTCAA
Hit: AAGCTATACCAGTTTGGAGCAT
Hit: AAGCTATACCATAGCCTCCGTA
Hit: AAGCTATACCATCCGTGGTTGC
Hit: AAGCTATACCATCCGTGGTTGC
Hit: AAGCTATACCATTTCGATGACG
Hit: AAGCTATACCATTTGCCCAGCT
Hit: AAGCTATACCATTTGCCCAGCT
Hit: AAGCTATACCATTTGCCCAGCT
Hit: AAGCTATACCCAAACAAGAAGG
Hit: AAGCTATACCCAAATGACGGGC
Hit: AAGCTATACCCAAATGACGGGC
Hit: AAGCTATACCCAAGTGTCTGTA
Hit: AAGCTATACCCAAGTGTCTGTA
Hit: AAGCTATACCCAAGTGTCTGTA
Hit: AAGCTATACCCAGAAGGTTGTT
Hit: AAGCTATACCCAGAA

Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCACGTCACTTA
Hit: AAGGGTGAAGCCATCTCTGGGA
Hit: AAGGGTGAAGCCATCTCTGGGA
Hit: AAGGGTGAAGCCCATCTGGAGA
Hit: AAGGGTGAAGCCCATCTGGAGA
Hit: AAGGGTGAAGCCCATCTGGAGA
Hit: AAGGGTGAAGCCGTGAAAGACC
Hit: AAGGGTGAAGCCTCGTCAGAAC
Hit: AAGGGTGAAGCCTCGTCAGAAC
Hit: AAGGGTGAAGCCTCGTCAGAAC
Hit: AAGGGTGAAGCCTCGTCAGAAC
Hit: AAGGGTGAAGCCTCGTCAGAAC
Hit: AAGGGTGAAGCCTGTAGTGGCA
Hit: AAGGGTGAAGCGAGAATCGACG
Hit: AAGGGTGAAGCGAGAATCGACG
Hit: AAGGGTGAAGCGCCACCATTTC
Hit: AAGGGTGAAGCGCCACCATTTC
Hit: AAGGGTGAAGCGCGAGAGCTAT
Hit: AAGGGTGAAGCGGAACTATACT
Hit: AAGGGTGAAGCGTGTACTGAAT
Hit: AAGGGTGAAGCTAATCCAGATG
Hit: AAGGGTGAAGCTATCCCATCCG
Hit: AAGGGTGAAGCTATCCCATCCG
Hit: AAGGGTGAAGCTCACAAGCCTA
Hit: AAGGGTGAAGCTCATCTACGTC
Hit: AAGGGTGAAGCTCATCTACGTC
Hit: AAGGGTGAAGCTCATCTACGTC
Hit: AAGGGTGAAGCTCATCTACGTC
Hit: AAGGGTGAAGCTCAT

KeyboardInterrupt: 

In [28]:
adata.uns[]#.at["Human_brain_1_1+AGCGATAGAACCAGGTAATGACGTGTACTGAC", ]

OverloadedDict, wrapping:
	OrderedDict()
With overloaded keys:
	['neighbors'].

In [ ]:
cell_types = []
for ct in adata.obs['cell type']:
    if ct not in cell_types:
        cell_types.append(ct)

len(cell_types)
cell_types

In [ ]:
frag = pysam.TabixFile(WORKING_DIR + 'frag/colon_sigmoid_SM-JF1O8_rep1_fragments.bed.gz')